In [ ]:
import glob
import os
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import tqdm
import h5py
import yaml

from astropy.modeling.blackbody import blackbody_lambda, blackbody_nu
from astroquery.vizier import Vizier

from thejoker import JokerSamples
from twoface.ext import FileLoader
from twoface.mass import asini, a2sini, stellar_radius, get_m2_min, mf

In [ ]:
cache_path = '../data/candidates/cache'

Catalogs we should query:
* Galex
* PS1
* WISE

In [ ]:
sheet = FileLoader.GoogleSheets('1z0ukn8QWJL7dTrPdCFWlhQ6r-X-kwiA4z0YRqalZsY4', 1633658731)
candidates = sheet.load()

In [ ]:
allstar = fits.getdata('/Users/adrian/data/APOGEE_DR15beta/allStar-t9-l31c-58158.fits')
rc = fits.getdata('/Users/adrian/data/APOGEE_DR14/apogee-rc-DR14.fits')
ness = fits.getdata('/Users/adrian/projects/twoface/data/NessRG.fits')

allstar = Table(allstar[np.isin(allstar['APOGEE_ID'], candidates['APOGEE_ID'])])
rc = Table(rc[np.isin(rc['APOGEE_ID'], candidates['APOGEE_ID'])])

ness = Table(ness[np.isin(ness['2MASS'], candidates['APOGEE_ID'])])
ness.rename_column('2MASS', 'APOGEE_ID')

In [ ]:
c = coord.SkyCoord(ra=allstar['RA']*u.deg, dec=allstar['DEC']*u.deg)
allstar['l'] = c.galactic.l.degree
allstar['b'] = c.galactic.b.degree
allstar['ecl_l'] = c.barycentrictrueecliptic.lon.degree
allstar['ecl_b'] = c.barycentrictrueecliptic.lat.degree

In [ ]:
tmp = join(candidates[['APOGEE_ID']], allstar, join_type='left', keys='APOGEE_ID')
_, idx = np.unique(tmp['APOGEE_ID'], return_index=True)
tmp = tmp[idx][['APOGEE_ID', 'RA', 'DEC', 'l', 'b', 'ecl_l', 'ecl_b', 
                'LOGG', 'LOGG_ERR', 'TEFF', 'TEFF_ERR', 'M_H', 'M_H_ERR']]
assert np.all(tmp['APOGEE_ID'] == sorted(candidates['APOGEE_ID']))

tmp = join(tmp, rc[['APOGEE_ID', 'RC_DIST']], join_type='left', keys='APOGEE_ID')
tmp = join(tmp, ness[['APOGEE_ID', 'lnM', 'lnAge', 'e_logM', 'e_logAge']], 
           join_type='left', keys='APOGEE_ID')

In [ ]:
tmp.write('../data/tmp.csv')

In [ ]:
for col in tmp.colnames:
    tmp[col][tmp[col] == -9999] = np.nan

In [ ]:
v = Vizier(columns=["*", "+_r"], catalog="II/349/ps1")

In [ ]:
c = coord.SkyCoord(ra=line['RA']*u.deg, dec=line['DEC']*u.deg)

In [ ]:
line

In [ ]:
c

In [ ]:
for tbl in v.query_region(c, radius=2*u.arcsec):
    print(tbl)

In [ ]:
tmp

In [ ]:
x = np.linspace(2000*u.Angstrom, 5*u.micron, 10000)
Blambda = blackbody_lambda(x, 5500*u.Kelvin)

In [ ]:
plt.plot(x, x*Blambda, marker='')

In [ ]:
# from astroquery.mast import Catalogs
# mast_tbl = Catalogs.query_region("83.46395 26.425833", radius=10*u.arcsec)
# mast_tbl

In [ ]:
job = tap.launch_job('''SELECT TOP 100 * FROM "II/312/ais" WHERE 1=CONTAINS(POINT('ICRS',"II/312/ais".RAJ2000,"II/312/ais".DEJ2000), CIRCLE('ICRS', {0.ra.degree}, {0.dec.degree}, 10/3600.))'''.format(c))

In [ ]:
job.get_results()

In [ ]:
for c in all_c:
    mast_tbl = Catalogs.query_region(c, radius=10*u.arcsec)
    break

In [ ]:
Catalogs.q

In [ ]:
c.ra.degree, c.dec.degree

In [ ]:
mast_tbl